# Speeding up python code

Python offers many advantages for scientific computation:

* It is very easy to implement new ideas, with a time of coding that usually is much smaller than in a low-level language such as fortran or C

* There are many modules implementing common algorithms, even in the scientific domain, which can save a lot of time in coding

* Given that it is interpreted, it offers a more natural environment (specially aided with Ipython) for scientific exploration

It has however a severe drawback

* Due to the very fact that it is interpreted, it is much slower, sometimes an order of magnitude, than the corresponding algorithm coded in a compiled language

There are different alternatives to speed up Python code

As a rule of thumb, use numpy/scipy always if your algorithm can be expressed in vectorial form

Some algorithms, however, cannot be expressed in vectorial form, for example when they involve looking at positions in an array in non sequential ways. 

One of the main causes of the slow performance of python is its lack of static typing
The fact that variables do not have a predefided type makes coding faster and easier, but it imposes an computation overload, since the type of each variable must be inferred at execution time
In compiled languages, the stric type of operations allows for a large number of optimizations, that are performed at compile time, and do not get reflected when excuting the code.

Two different approaches can be applied to alleviate this problem:


* Code specific parts of the algorithm directly in C/fortran and link them into the Python source code (or access a C/Fortran library to use some specific functions)

* Somehow, compile the python code to make it ran faster
    - This can be done appliying "Just-in-time" (JIT) technologies using a Low-level virtual machine (LLVM). The first time the code (usually in the form of a function) is executed, it becomes compiled, and successive executions are performed on the compiled object, achieving thus better performance.
    
The first approach is quite useful, since it allows to access legacy code write (and thorougly tested) in Fortran, which we do not have to reinvent (BLAS, LAPACK, etc)

Several alternative approaches are possible:

* F2py (you have already seen)
* Numba
* Cython
* Ctypes


## Numba

Numba is, by far, the simplest way to jit-compile python code. One can achive this, in the simplest case, by just adding a **decorator** on top of the definition of a function. The performance of the jit-compiled code can be improved by adding additional information of the types of the parameters and output of the function, but in many cases it is not necessary.

Let us see numba in action.

### An example: The Mandelbrot set

Let us consider a classical example, the generation of the Mandelbrot set, which is defined as the set of points in the complex plane that does not approach infinity under the iterative map

$$
z_{n+1} = z_n^2 + c
$$

That is, a complex number $c$ is part of the Mandelbrot set if, starting form $z_0 = 0$, the application of the iteration leaves $z_n$ bounded for any large value of $n$. 

One way to depict it graphically is using the **escape time algorithm**: Fixing a number of interations, for every point in the complex plane one checks for which value of $m$ the absolute value of $z_{m}$ becomes larger than a given threshold. Then the color associated to the complex number is given by $m$. 

Let us code the generation of the mandelbrot set in pure python

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# create the mandelbrot set for a region of the complex plane between maximum and minimum x,y
# the color codes are stored in a two dimensional array of pixels
def create_mandelbrot_python(min_x, max_x, min_y, max_y, image, iters):
    height = image.shape[0]
    width = image.shape[1]

    pixel_size_x = (max_x - min_x) / width
    pixel_size_y = (max_y - min_y) / height
    for x in range(width):
        real = min_x + x * pixel_size_x
        for y in range(height):
            imag = min_y + y * pixel_size_y
            color = color_mandelbrot_python(real, imag, iters)
            image[y, x] = color

    return image

# define the color of each pixel according to the escape time algorithm
def color_mandelbrot_python(x, y, max_iters):
    i = 0
    c = complex(x, y)
    z = 0.0j
    for i in range(max_iters):
        z = z**2 + c
        if abs(z)**2 >= 4:
            return i

    return 255

In [ ]:
# create the image and plot it
image = np.zeros((500, 750), dtype=int)
# imshow shows an image encoded in colors for each pixel
m = create_mandelbrot_python(-1.75, 1.0, -1.0, 1.0, image, 20)
plt.imshow(np.log(m),cmap=plt.cm.hot)
plt.xticks([]); plt.yticks([])

Let us time the excution of the algorithm

In [ ]:
%timeit create_mandelbrot_python(-1.75, 1.0, -1.0, 1.0, image, 20)

Let us now implement a numba version. The only work to do is to add the `@jit` decorator to the function definitions

In [ ]:
# import jit form numba
from numba import jit

In [ ]:
@jit
def color_mandelbrot_numba(x, y, max_iters):
    i = 0
    c = complex(x, y)
    z = 0.0j
    for i in range(max_iters):
        z = z**2 + c
        if abs(z)**2 >= 4:
            return i

    return 255

@jit
def create_mandelbrot_numba(min_x, max_x, min_y, max_y, image, iters):
    height = image.shape[0]
    width = image.shape[1]

    pixel_size_x = (max_x - min_x) / width
    pixel_size_y = (max_y - min_y) / height
    for x in range(width):
        real = min_x + x * pixel_size_x
        for y in range(height):
            imag = min_y + y * pixel_size_y
            color = color_mandelbrot_numba(real, imag, iters)
            image[y, x] = color

    return image

In [ ]:
# create the image and plot it
m = create_mandelbrot_numba(-1.75, 1.0, -1.0, 1.0, image, 20)
plt.imshow(np.log(m),cmap=plt.cm.hot)
plt.xticks([]); plt.yticks([])

In [ ]:
%timeit create_mandelbrot_numba(-2.0, 1.0, -1.0, 1.0, image, 20)

Amazingly huge gain for a tiny effort!!

### Never never understimate the power of numpy

In [ ]:
def create_mandelbrot_numpy(min_x, max_x, min_y, max_y, image, iters):
    height = image.shape[0]
    width = image.shape[1]
    
    divtime = iters + np.zeros((height, width), dtype=int)

    y, x = np.ogrid[ min_y:max_y:height*1j, min_x:max_x:width*1j ]
    
    c = x + y*1j
    z = 0.0
    
    for i in range(iters):
        z  = z**2 + c
        diverge = z*np.conj(z) >= 2**2            # who is diverging
        div_now = diverge & (divtime==iters)  # who is diverging now
        divtime[div_now] = i                  # note when
        z[diverge] = 2                        # avoid diverging too much

    return divtime
    

In [ ]:
%timeit create_mandelbrot_numpy(-2.0, 1.0, -1.0, 1.0, image, 20)

Faster than pure python, but slower than numba.

And the result, for some reason, is way much nicer ...

In [ ]:
m = create_mandelbrot_numpy(-1.75, 1.0, -1.0, 1.0, image, 20)
plt.imshow(np.log(m),cmap=plt.cm.hot)
plt.xticks([]); plt.yticks([])

## Profiling python code

Numba is trivially simple to apply, but beware:

*** Numba does not always produce such spectacular results ***

... although it can be fine tuned a but more

Sometimes we will need to use more complex optimization methods.

But before using them, we must profile our code, to pinpoint the particular places where it is slower. We should then only optimize those bottlenecks.

Remember: "Premature optimization is the root of all evil"



Python, and IPython, offer several alternatives for profiling code

### `%timeit`

To compute the time taken by a single statement or a short calculatiion, we can bechmark them by hand using the `%timeit` and `%%timeit` line and cell magic commands, as we have already seen

### `cProfile`

`cProfile` is a Python module that allows to compute the time spent by every function called in a Python program

We can easiy use it in an interactive sassions with IPython, using the `%%prun` magic


In [ ]:
%%prun -s cumulative -q -l 10 -T Prof1

### Command arguments:
### -s : Sort functions by time used
### cumulative : Create a cumulative report of the time spent in each function
### -q : Supress the pager output (cleaner output)
### -l n : Limit the numbner of lines displayed by function
### -T "filename" : Save the report on a file

create_mandelbrot_python(-1.75, 1.0, -1.0, 1.0, image, 20)

In [ ]:
# read the report with %more magic
!more Prof1

### `line_profiler`

`cProfile` profiles a Python code at the level of functions. If our code is modularized, this is the best option. But sometimes we need a more fine-grained information, at the level of the lines executed by the code. The module `line_profiler` performs this task.

Unfortunately, it does not belong to the standard Python installation. It must be installed using the command

    pip install line_profiler
    
Using it in Ipython is easy, with the extension `line_profiler`, that we have to load in out notebook session

In [ ]:
%load_ext line_profiler

Now we can use it with the magic `lprun`

In [ ]:
# Calling it
### Syntax:
### -T "filename" : Store results in file "filename:
### -f function : function to profile
%lprun -s  -T lprof0 -f create_mandelbrot_python create_mandelbrot_python(-1.75, 1.0, -1.0, 1.0, image, 20)

Look at a function inside, that looks costly

In [ ]:
# Calling it
### Syntax:
### -T "filename" : Store results in file "filename:
### -f function : function to profile
%lprun -s  -T lprof0 -f color_mandelbrot_python create_mandelbrot_python(-1.75, 1.0, -1.0, 1.0, image, 20)

## Optimizing code by writing parts in other high efficiency language: Fortran or C

### f2py

`f2py` is a program that automatically wraps fortran code to use it into Python. We can compile fortran code and embed it into a module that we can then import into a Python program.

`f2py` is part of numpy, so it can easily interact with it.


In [ ]:
import random

In [ ]:
# Example 
def scalar_product_python(x, y):
    scalar = 0.0
    for i in xrange(len(x)):
        scalar += x[i]*y[i]
        
    return scalar

In [ ]:
N = 1000000
x = [random.random() for i in xrange(N)]
y = [random.random() for i in xrange(N)]

%timeit scalar_product_python(x, y)

Write the function in Fortran (attention old fortran 77 style only!!!)
We use the `%%write` magic to write a file

In [ ]:
%%file prod.f
      real function prod(x, y, n)
      real x(n)
      real y(n)
      integer, intent(in) :: n

                
      prod = 0.0
      do i=1,n
         prod = prod + x(i)*y(i)
      enddo
      
      end function prod
        
            

In [ ]:
# compite the file into fortran, and create a module
!f2py -c prod.f -m prodf

In [ ]:
# import the module
import prodf

In [ ]:
%timeit prodf.prod(x, y, N)

Another way, using Ipython magic

Attention, you hve to install 

    pip install fortran-magic
    

In [ ]:
%load_ext fortranmagic

In [ ]:
%%fortran

real function prod2(x, y, n)
      real x(n)
      real y(n)
      integer, intent(in) :: n

                
      prod2 = 0.0
      do i=1,n
         prod2 = prod2 + x(i)*y(i)
      enddo
      
      end function prod2


In [ ]:
%timeit prod2(x, y, N)

In [ ]:
# Curiosity: In numpy?
import numpy as np
x = np.array(x)
y = np.array(y)

In [ ]:
%timeit (x*y).sum()

Numpy seems better, but we have cheated here: `f2py` is integrated with numpy, and works beter with numpy arrays

In [ ]:
# now x and y are numpy arrays
%timeit prod2(x, y, N)

Same as numpy, or even faster. 

Notice, however, that we are still cheating again, since the function implements something that can be done vectorially, and vectorial operations are very fast in numpy: they are coded directly in C. Something not vectorial will perform badly in numpy and much better in Fortran

### `Cython`

`Cython` is both a language (a superset of Python) and a library. It allows to start with pure python, and add annotations about the type of the variables. Cython translates the python code into C and compiles it into a module that we can later use into any python program

In [ ]:
# first, load the ipython extension
%load_ext cython

In [ ]:
%%cython

def c_scalar_product_python(x, y):
    scalar = 0.0
    for i in xrange(len(x)):
        scalar += x[i]*y[i]
        
    return scalar


In [ ]:
N = 1000000
x = [random.random() for i in xrange(N)]
y = [random.random() for i in xrange(N)]

In [ ]:
%timeit scalar_product_python(x,y)

In [ ]:
%timeit c_scalar_product_python(x,y)

In [ ]:
x = np.array(x)
y = np.array(y)

In [ ]:
%timeit c_scalar_product_python(x,y)

Some improvement, but the performance is dismal with numpy arrays. We can improve even more, by adding anotations on the types, specially when using numpy arrays

In [ ]:
%%cython
cimport numpy as np
def c_scalar_product_python_2(np.ndarray[np.float64_t, ndim=1] x, np.ndarray[np.float64_t, ndim=1] y):
    cdef float scalar = 0.0
    cdef int i
    for i in xrange(len(x)):
        scalar += x[i]*y[i]
        
    return scalar

In [ ]:
%timeit c_scalar_product_python_2(x,y)

Again, no match for numpy, but we can improve a lot probles that do not yield easily to numpy.

Additionally, observe that the improvement is the same as in the pure Fortran function, but the effort to write it is minimal